In [11]:
from openai import OpenAI
# Set OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:1122/v1"
import time

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)


chat_response = client.chat.completions.create(
    model="quant_lora",
    messages=[
        {"role": "system", "content": "你是一个人工智能助手."},
        {"role": "user", "content": "告诉我有关北京的特产"},
    ]
)
print("Chat response:", chat_response)

Chat response: ChatCompletion(id='cmpl-3280735e31e54a6a856eae49efec5235', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='北京的特产有“北京烤鸭”、“豆汁焦圈”、“羊蝎子”、“艾窝窝”、“冰糖葫芦”等，它们都是北京独有的特色美食。此外，还有“景泰蓝”、“北京绢人”、“雕漆”等工艺品，以及“瓷器”、“茶叶”等传统文化产品。', role='assistant', function_call=None, tool_calls=None), stop_reason=None)], created=1715767771, model='quant_lora', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=71, prompt_tokens=23, total_tokens=94))


In [1]:
import os

os.environ["cuda_visible_devices"] ='0, 5'

In [1]:
from vllm import LLM, SamplingParams
import time


# Sample prompts.
prompts = [
    "告诉我有关北京的特产",
] * 5

# Create a sampling params object.
sampling_params = SamplingParams(temperature=0.8, top_p=0.95, max_tokens=2000)

# Create an LLM.
llm = LLM(model='gptq_qwen/',quantization='gptq',
         trust_remote_code=True)

prompt_template='''
给定一句话：“%s”，请你按步骤要求工作。

步骤1：识别这句话中的城市和日期共2个信息
步骤2：根据城市和日期信息，生成JSON字符串，格式为{"city":城市,"date":日期}

请问，这个JSON字符串是：
'''

Q_list=['2020年4月16号三亚下雨么?','青岛3-15号天气预报','5月6号下雪么, 城市是威海','青岛2023年12月30号有雾霾么?','我打算6月1号去北京旅游，请问天气怎么样？','你们打算1月3号坐哪一趟航班去上海？','小明和小红是8月8号在上海结婚么?',
        '一起去东北看冰雕么, 大概是1月15号左右, 我们3个人一起']

prompts = [prompt_template%(Q,) for Q in Q_list]

start = time.time()
outputs = llm.generate(prompts, sampling_params)
print(time.time()-start)


WARNING 05-15 17:48:44 config.py:211] gptq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 05-15 17:48:44 llm_engine.py:74] Initializing an LLM engine (v0.4.0.post1) with config: model='gptq_qwen/', tokenizer='gptq_qwen/', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=True, quantization=gptq, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=0)
WARNING 05-15 17:48:44 tokenizer.py:104] Using a slow tokenizer. This might cause a significant slowdown. Consider using a fast tokenizer instead.
INFO 05-15 17:48:48 selector.py:16] Using FlashAttention backend.
INFO 05-15 17:48:55 model_runner.py:104] Loading model weights took 5.5044 GB
INFO 05-15 17:48:57 gpu_executor.py:94] # GPU blocks: 3594, # CPU blocks: 512
INFO 05-15 17:49:04 model_runner.py:791] Capturing th

Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:26<00:00,  5.34s/it]


26.685515642166138


Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:30<00:00,  3.85s/it]

30.81380271911621


In [2]:
prompts = [
    "告诉我有关上海的特产",
] * 100


start = time.time()
outputs = llm.generate(prompts, sampling_params)
num_tokens = sum([len(output.outputs[0].token_ids) for output in outputs]) 
spend_time = time.time() - start
print(num_tokens / spend_time )


Processed prompts:   0%|                                                                                                                           | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [7]:
#  tp    1     2
# 1     71     
# 10    224
# 100   894
# 500   1825
# 1000  2022   2042
# 10000 2258

# 张量并行数量增大会明显降低显存占用，但是不会增大推理速度，甚至略微降低
#       chat  AWQ(1.5) GPTQ(1.5)  MyGPTQ    GPTQ(1)
# 1     71      100    134     135
# 10    224     324    360     473
# 100   894     910    980     827           827           
# 500   1825    1553   1526    858
# 1000  2022    1642   1828
# 10000 2258    1603   2159

# batch size 小于100时，推理速度受限于内存中传输权重的带宽，量化模型较小所以可以更快
# batch size 大时，推理速度受限于计算，量化模型的反量化操作增加了计算资源，推理速度更慢

1732

In [ ]:
outputs[0].outputs[0].text

In [26]:
str = '北京的特产有“北京烤鸭”、“豆汁焦圈”、“羊蝎子”和“冰糖葫芦”等，其中“北京烤鸭”是最具代表性的，它以其独特的烹饪技术和美味而闻名；“豆汁焦圈”是一种传统的北京小吃，豆汁是用绿豆发酵而成，焦圈是用面粉和糖制成；“羊蝎子”是一种独特的烹饪方法，它使用羊的脊髓，经过烹饪后变得鲜美可口；“冰糖葫芦”是一种以山楂、葡萄、苹果等水果串起来，外面包裹冰糖的特色小吃，口感酸甜可口。'

from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(
    '/data/liucd/langchain/Qwen/Qwen1___5-7B-Chat-AWQ', # path to the output directory
    trust_remote_code=True
)

out = tokenizer(str)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [29]:
len(out['input_ids'])

134

In [31]:
out

{'input_ids': [68990, 9370, 112984, 18830, 2073, 68990, 102554, 105397, 854, 91956, 99955, 102514, 100479, 100044, 854, 91956, 101187, 116558, 44729, 854, 33108, 2073, 100038, 100443, 111065, 854, 49567, 3837, 90919, 2073, 68990, 102554, 105397, 854, 104890, 76813, 99661, 104196, 3837, 99652, 109859, 105071, 111611, 107772, 106800, 68536, 109836, 24968, 2073, 99955, 102514, 100479, 100044, 854, 101158, 105062, 68990, 107600, 3837, 99955, 102514, 20412, 11622, 118693, 108279, 106042, 3837, 100479, 100044, 20412, 11622, 112398, 33108, 100443, 108711, 24968, 2073, 101187, 116558, 44729, 854, 101158, 105071, 111611, 39907, 3837, 99652, 37029, 101187, 9370, 109257, 103661, 3837, 101897, 111611, 33447, 101197, 99705, 57566, 30440, 39426, 24968, 2073, 100038, 100443, 111065, 854, 101158, 23031, 57811, 121142, 5373, 101580, 5373, 104167, 49567, 104618, 51575, 99793, 3837, 102586, 108232, 100038, 100443, 9370, 100175, 107600, 3837, 107816, 99918, 100475, 30440, 39426, 1773], 'attention_mask': [

In [57]:
!curl http://localhost:1212/v1/completions \
-H "Content-Type: application/json" \
-d '{
"model": "facebook/opt-125m",
"prompt": "San Francisco is a",
"max_tokens": 7,
"temperature": 0
}'

SyntaxError: unmatched '}' (3344056460.py, line 6)